In [19]:
import sys
import os
sys.path.append(os.path.abspath(".."))

# for numeric calculations
import numpy as np

# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Import the DstApi utility class
from dstapi.dstapi import DstApi

In [20]:
# Initialize the class with the target table
table_name = 'RECIDIV6'
data_table = DstApi(table_name)

# Get the table summary
data_table.tablesummary(language='da')

Table RECIDIV6: Personer efter køn, alder, uddannelse, varighed til tilbagefald og tid
Last update: 2024-11-07T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,KØN,3,TOT,I alt,K,Kvinder,False
1,ALDER1,9,00,I alt,6099,60 år og derover,False
2,UDDANNELSE,6,TOT,I alt,00,Uoplyst uddannelse,False
3,VARTILBAGE,5,00,I alt,1324,Efter 1 år og indenfor 2 år,False
4,Tid,13,2008:2010,2008:2010,2020:2022,2020:2022,True


In [21]:
# Start by constructing a basic dictionary
params = data_table.define_base_params(language='da')
params['format'] = 'csv'

params

{'table': 'recidiv6',
 'format': 'csv',
 'lang': 'da',
 'variables': [{'code': 'KØN', 'values': ['*']},
  {'code': 'ALDER1', 'values': ['*']},
  {'code': 'UDDANNELSE', 'values': ['*']},
  {'code': 'VARTILBAGE', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [22]:
# Variable exclusion rules
exclusions = {
    '*': ['I alt']  # Will be applied to all variables in addition to specific ones
}

In [23]:
for var in params['variables']:
    # Extract all possible values for this column
    levels = data_table.variable_levels(var['code'], language='da')

    # Gather exclusions: specific + global
    exclude_texts = set(exclusions.get(var['code'], [])) | set(exclusions.get('*', []))

    # Exclude specified text values
    levels = levels[~levels['text'].isin(exclude_texts)]

    # Assign the filtered id values
    var['values'] = levels['id'].tolist()

In [24]:
params

{'table': 'recidiv6',
 'format': 'csv',
 'lang': 'da',
 'variables': [{'code': 'KØN', 'values': ['M', 'K']},
  {'code': 'ALDER1',
   'values': ['1519', '2024', '2529', '3034', '3539', '4049', '5059', '6099']},
  {'code': 'UDDANNELSE', 'values': ['10', '20', '35', '40', '00']},
  {'code': 'VARTILBAGE', 'values': ['0000', '0006', '0712', '1324']},
  {'code': 'Tid',
   'values': ['2008:2010',
    '2009:2011',
    '2010:2012',
    '2011:2013',
    '2012:2014',
    '2013:2015',
    '2014:2016',
    '2015:2017',
    '2016:2018',
    '2017:2019',
    '2018:2020',
    '2019:2021',
    '2020:2022']}]}

In [25]:
# Fetch all the data from the table with the specified params
df = data_table.get_data(params=params)

In [26]:
df.shape

(4160, 6)

In [27]:
df.sample(5)

,KØN,ALDER1,UDDANNELSE,VARTILBAGE,TID,INDHOLD
3683,Kvinder,50-59 år,Grundskole,Efter 1 år og indenfor 2 år,2012:2014,97
914,Mænd,30-34 år,Erhvervsuddannelse,Efter 6 måneder og indenfor 1 år,2012:2014,366
3838,Kvinder,50-59 år,Videregående uddannelse,Efter 1 år og indenfor 2 år,2011:2013,78
3059,Kvinder,30-34 år,Videregående uddannelse,Efter 1 år og indenfor 2 år,2012:2014,48
1360,Mænd,40-49 år,Gymnasial uddannelse,Ingen tilbagefald,2016:2018,988


In [28]:
# Construct the path
directory = f"../Data/{table_name}"
filename = f"Cleaned{table_name}.csv"
filepath = os.path.join(directory, filename)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the DataFrame
df.to_csv(filepath, index=False)

In [29]:
# Select all the columns that are not numeric
data_column_category = df.select_dtypes(exclude=[np.number]).columns

In [30]:
# Select all the columns that are not numeric
data_column_category = df.select_dtypes(exclude=[np.number]).columns
# View column categories
data_column_category

Index(['KØN', 'ALDER1', 'UDDANNELSE', 'VARTILBAGE', 'TID'], dtype='object')

In [31]:
# Get a sample from the datafrane with only the categorical columns
df[data_column_category].sample(5)

,KØN,ALDER1,UDDANNELSE,VARTILBAGE,TID
1045,Mænd,35-39 år,Grundskole,Ingen tilbagefald,2013:2015
3895,Kvinder,50-59 år,Uoplyst uddannelse,Efter 1 år og indenfor 2 år,2016:2018
1141,Mænd,35-39 år,Gymnasial uddannelse,Efter 1 år og indenfor 2 år,2018:2020
3393,Kvinder,40-49 år,Grundskole,I løbet af 6 måneder,2008:2010
1083,Mænd,35-39 år,Grundskole,Efter 1 år og indenfor 2 år,2012:2014


In [32]:
label_encoder = LabelEncoder()

In [33]:
# Make a copy of the DataFrame
df_label_encoded = df.copy()

In [34]:
# Performing label encoding
for i in data_column_category:
    df_label_encoded[i] = label_encoder.fit_transform(df_label_encoded[i])

In [35]:
# View the label encoded data
print("Label Encoded Data: ")
df_label_encoded.sample(5)

Label Encoded Data: 


,KØN,ALDER1,UDDANNELSE,VARTILBAGE,TID,INDHOLD
1633,1,6,2,2,8,80
1008,1,3,3,2,7,159
700,1,2,4,2,11,171
1257,1,4,3,3,9,548
1772,1,6,3,3,4,608


In [36]:
# Construct the path
directory = f"../Data/{table_name}"
filename = f"Encoded{table_name}.csv"
filepath = os.path.join(directory, filename)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the DataFrame
df_label_encoded.to_csv(filepath, index=False)